## ДЗ №2. Матричные факторизации

#### В этой домашке вам предстоит реализовать некоторые базовые модели матричной факторизации

#### Дата выдачи: 17.02.25

#### Мягкий дедлайн: 02.03.25 23:59 MSK

#### Жесткий дедлайн: 09.03.25 23:59 MSK

В этом задании мы будем работать с классическим для рекоендательных систем датасетом [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/). Датасет содержит рейтинги оценки для 4000 фильмов от 6000 пользователей. Более подробное описание можете найти на странице с датасетом и в README файле

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!cat ml-1m/README

--2025-03-02 11:15:09--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  27.6MB/s    in 0.2s    

2025-03-02 11:15:09 (27.6 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be u

In [2]:
import pandas as pd
import numpy as np
from typing import Union

In [3]:
df = pd.read_csv("ml-1m/ratings.dat", sep='::', names=['user_id', 'item_id', 'rating', 'timestamp'], engine='python')
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df.drop('timestamp', axis=1, inplace=True)
df.head()

,user_id,item_id,rating,datetime
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [4]:
value_counts = df['item_id'].value_counts()
filtered_values = value_counts[value_counts > 20].index
df = df[df['item_id'].isin(filtered_values)].copy()

In [5]:
train_end = '2000-12-01'
df_train = df[df['datetime'] < train_end].copy()
df_test = df[df['datetime'] >= train_end].copy()
df_train.shape, df_test.shape

((787420, 4), (207432, 4))

In [6]:
train_users = df_train['user_id'].unique()
train_items = df_train['item_id'].unique()

df_test = df_test[df_test['user_id'].isin(train_users)]
df_test = df_test[df_test['item_id'].isin(train_items)]
df_test.shape

(106471, 4)

In [7]:
from sklearn.preprocessing import LabelEncoder

user_le = LabelEncoder()
item_le = LabelEncoder()

df_train['user_id'] = user_le.fit_transform(df_train['user_id'])
df_train['item_id'] = item_le.fit_transform(df_train['item_id'])

df_test['user_id'] = user_le.transform(df_test['user_id'])
df_test['item_id'] = item_le.transform(df_test['item_id'])

In [8]:
df_train['user_id'].nunique(), df_train['user_id'].max()
df_train['item_id'].nunique(), df_train['item_id'].max()

(3010, 3009)

In [9]:
df_train

,user_id,item_id,rating,datetime
100409,0,2994,3,2000-11-30 23:49:23
100411,0,929,4,2000-11-30 23:52:33
100412,0,567,4,2000-11-30 23:51:54
100415,0,3005,1,2000-11-30 23:58:06
100416,0,3006,4,2000-11-30 23:57:50
...,...,...,...,...
1000204,5364,814,1,2000-04-26 02:35:41
1000205,5364,817,5,2000-04-25 23:21:27
1000206,5364,478,5,2000-04-25 23:19:06
1000207,5364,819,4,2000-04-26 02:20:48


##### Задание 1. Напишем функцию, которая превратит датафрейм в матрицу интеракций. В функции df_to_matrix реализуйте функцию, которая принимает датафрейм и возвращает np.array матрицу интеракций. В функции df_to_coo реализуйте функцию, которая принимает датафрейм и возвращает разреженную матрицу интеракций в coo_array формате

In [179]:
def df_to_matrix(df: pd.DataFrame) -> np.ndarray:
    df = df.copy()

    num_users = df['user_id'].nunique()
    num_items = df['item_id'].nunique()

    result = np.zeros((num_users, num_items), dtype=int)

    for _, row in df.iterrows():
        user_idx = row['user_id']
        item_idx = row['item_id']
        result[user_idx, item_idx] = row['rating']

    return result #shape ~ [n_users, n_items]

In [180]:
interactions = df_to_matrix(df_train)

In [12]:
from scipy.sparse import coo_array

def df_to_coo(df: pd.DataFrame) -> coo_array:
    df = df.copy()

    num_users = df['user_id'].nunique()
    num_items = df['item_id'].nunique()

    result = coo_array(
        (df['rating'], (df['user_id'], df['item_id'])),
        shape=(num_users, num_items)
    )

    return result # coo_array

In [13]:
coo_interactions = df_to_coo(df_train)

In [14]:
assert (interactions != 0).sum() == df_train.shape[0]
assert interactions[0, 2994] == 3
assert interactions[2369, 1203] == 5
assert interactions[1557, 459] == 3
assert np.allclose(coo_interactions.toarray(), interactions)

##### Задание 2.1. Рассмотрим [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition). Возьмите готовую реализуцию алгоритма из numpy.linalg или из scipy.linalg и примените алгоритм к матрицам интеракций, полученным в первом задании. Для работы со sparse матрицей обычная реализация svd не подойдет и нужно будет воспользоваться scipy.sparse.linalg.svds. Вам нужно разложить матрицу интеракций на 3 матрицы U, S, V, а затем перемножить их и восстановить изначальную матрицу. При полном разложении исходная матрица должна восстанавливаться максимально хорошо

In [15]:
from scipy.sparse.linalg import svds # для работы со sparse матрицей
from numpy.linalg import svd

def make_svd(interactions: Union[np.ndarray, coo_array], n_singular_values: int = -1):
    # функция должна работать и для полной матрицы и для sparse матрицы (вам поможет isinstance).
    # если n_singular_values = -1, то берем все сингулярные числа для полной матрицы
    # и все кроме одного сингулярного числа для coo-матрицы(иначе scipy.sparse.linalg.svds не будет работать)

    interactions = interactions.copy()
    # для полной матрицы!
    if isinstance(interactions, np.ndarray):
        if n_singular_values == -1:
            U, S, Vt = np.linalg.svd(interactions, full_matrices=False) # берем все сингулярные числа для полной матрицы
            S = np.diag(S)  # преобразуем в диагональную матрицу
        else:
            U, S, Vt = np.linalg.svd(interactions, full_matrices=False)
            U = U[:, :n_singular_values]
            S = np.diag(S[:n_singular_values])
            Vt = Vt[:n_singular_values, :]
    # для coo-матрицы!
    elif isinstance(interactions, coo_array):
        if n_singular_values == -1:
            n_singular_values = min(interactions.shape) - 1 # берём все кроме одного сингулярного числа для coo-матрицы
        interactions = interactions.astype(np.float64)
        U, s, Vt = svds(interactions, k=n_singular_values)
        S = np.diag(s)
        V = Vt.T # транспонуруем
    else:
        raise ValueError("Некорректный тип :(")

    return U, S, Vt

In [16]:
U, S, V = make_svd(interactions)
assert np.allclose(U @ S @ V, interactions)

In [17]:
U1, S1, V1 = make_svd(interactions, 10)
U, S, V = make_svd(coo_interactions, 10)
assert np.allclose(U1 @ S1 @ V1, U @ S @ V)

##### Задание 2.2. Теперь попробуем сделать рекомендации с помощью SVD. Мы научились восстанавливать исходную матрицу с помощью разложения, теперь же мы хотим порекомендовать пользователю айтемы, которые будут для него максимально релевантны(в восстановленной матрице у них будет самый высокий скор). Для каждого пользователя нужно будет найти индексы айтемов, которые имеют максимальный скор. При этом стоит обратить внимание, что мы не хотим рекомендовать пользователю айтемы, с которыми он уже взаимодействовал

In [18]:
def make_svd_recommendations(interactions: Union[np.ndarray, coo_array], n_singular_values: int = -1, top_k: int = 100):
    # Возвращает матрицу вида n_users, top_k, то есть для каждого пользователя возвращаем индексы
    # top_k самых релевантный айтемов среди тех с которыми он еще не взаимодействовал

    if isinstance(interactions, coo_array):
        interactions = interactions.toarray()

    U, S, V = make_svd(interactions, n_singular_values) # 1) применяем make_svd
    initial_matrix = U @ S @ V # 2) восстанавливаем изначальную матрицу

    n_users, n_items = interactions.shape
    recommendations = np.zeros((n_users, top_k), dtype=np.int64)

    for user_idx in range(n_users):
        # 3) получаем индексы айтемов, с которыми юзер уже взаимодействовал (!)
        already_touched_items = np.nonzero(interactions[user_idx, :])[0]
        # 4) сортируем айтемы по предсказанному рейтингу (лучшее наверх)
        item_scores = initial_matrix[user_idx, :]
        item_indices = np.argsort(item_scores)[::-1]

        # 5) возвращаем индексы top_k самых релевантный айтемов с которыми он еще не взаимодействовал
        recmmd_items = []
        for item_idx in item_indices:
            if item_idx not in already_touched_items:
                recmmd_items.append(item_idx)
            if len(recmmd_items) == top_k:
                break
        recommendations[user_idx, :] = recmmd_items

    return recommendations #shape ~ [n_users, top_k]

In [19]:
recs = make_svd_recommendations(interactions, -1, 100)
assert recs.shape == (interactions.shape[0], 100)

In [22]:
recs

array([[   1,    5,    2, ...,  710, 2279,   45],
       [   5,   32,   18, ..., 2886, 2417, 2776],
       [   1,   17,  299, ..., 1366, 2315, 2217],
       ...,
       [ 526,  137, 2489, ..., 2439,  234,  493],
       [ 996,   94, 2214, ..., 2699, 1497, 1650],
       [   3,    1,  118, ...,  417,    5,  280]])

##### Задание 2.3. Теперь давайте посмотрим как будет зависеть качетво рекомендаций, от количества сингулярных чисел, которые мы возьмем в SVD разложении. Переберите n_singular_values из списка [1, 10, 50, 200, 1000] и посмотрите как будет изменяться метрика NDCG на тестовом датасете для таких рекомендаций и как будет меняться время вычисления. Для каждого графики зависимости метрики NDCG от n_singular_values и времени работы алгоритма от n_singular_values(Время работы будет меняться только для sparse-матрицы, стоит запускать алгоритм именно для нее)

In [21]:
# метрики из семинара
def ndcg_metric(gt_items, predicted):
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)

    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_

def dcg(scores):
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )

def evaluate_recommender(df, model_preds, gt_col="test_interactions", topn=10):
    metric_values = []
    for idx, row in df.iterrows():
        gt_items = row[gt_col]
        metric_values.append(ndcg_metric(gt_items, row[model_preds]))

    return {"ndcg": np.mean(metric_values)}

In [22]:
import time
import plotly.express as px

def plot_graphs(interactions_matrix: Union[np.ndarray, coo_array], df_test, top_k=100):
    n_singular_values_list = [1, 10, 50, 200, 1000]
    ndcg_values = []
    time_values = []

    for n_singular_values in n_singular_values_list:
        # 1) старт замера
        start_time = time.time()
        recommendations = make_svd_recommendations(interactions_matrix, n_singular_values, top_k)
        end_time = time.time()
        time_values.append(end_time - start_time)

        user_ndcg_values = []
        for user_id in df_test['user_id'].unique(): # 2) проходимся по уникам в тесте
            if user_id < recommendations.shape[0]:
                gt_items = df_test[df_test['user_id'] == user_id]['item_id'].tolist() # 3) выводим айтемы для каждого юзера
                user_recommendations = recommendations[user_id]
                user_ndcg = ndcg_metric(gt_items, user_recommendations) # 4) считаем ndcg для каждого юзера
                user_ndcg_values.append(user_ndcg)

        avg_ndcg = np.mean(user_ndcg_values) if user_ndcg_values else 0 # 5) считаем средний ndcg
        ndcg_values.append(avg_ndcg)

    # 6) строим график зависимости ndcg vs n_singular_values
    fig_ndcg = px.line(x=n_singular_values_list, y=ndcg_values,
                       labels={'x': 'Singular Values', 'y': 'NDCG@100'},
                       title='NDCG@100 vs Singular Values')
    fig_ndcg.show()

    # 7) строим график зависимости time vs n_singular_values
    fig_time = px.line(x=n_singular_values_list, y=time_values,
                       labels={'x': 'Singular Values', 'y': 'Computation Time, sec'},
                       title='Computation Time vs Singular Values')
    fig_time.show()

In [23]:
plot_graphs(coo_interactions, df_test)

##### Задание 3.1. Перейдем к [ALS](http://yifanhu.net/PUB/cf.pdf). Возьмем реализацию iALS из библиотеки [implicit](https://benfred.github.io/implicit/api/models/cpu/als.html). Обучите ALS на нашем датасете, сделайте top_k рекомендации для юзеров из тестового датасета, и сравните метрики ALS с метриками, которые получились в SVD. Попробуйте перебрать гиперпараметры и найдите оптимальное число факторов, коэффициент alpha и коэффициент регуляризации.

In [25]:
# !pip install implicit

In [124]:
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

def make_als_recommendations(
    interactions: Union[np.ndarray, coo_array],
    top_k: int = 100,
    n_factors: int = 100,
    alpha: float = 1.0,
    regularization: float = 0.01,
):
    # преобразовываем в csr-формат! иначе не работает
    if isinstance(interactions, coo_array):
        interactions = csr_matrix(interactions)
    elif isinstance(interactions, np.ndarray):
        interactions = csr_matrix(interactions)
    else:
        raise ValueError("Некорректный тип данных. Ожидается np.ndarray или coo_array")

    als_model = AlternatingLeastSquares(factors=n_factors, regularization=regularization, alpha=alpha, iterations=20, random_state=42)
    als_model.fit(interactions)

    # генерим реки
    recommendations = als_model.recommend(
        np.arange(interactions.shape[0]),
        user_items=interactions,
        N=top_k,
        filter_already_liked_items=True
    )[0]

    return recommendations  #shape ~ [n_users, top_k]

In [125]:
recs_als = make_als_recommendations(interactions)
assert recs_als.shape == (interactions.shape[0], 100)

  0%|          | 0/20 [00:00<?, ?it/s]

In [122]:
recs_svd = make_svd_recommendations(interactions, n_singular_values=10, top_k=100)

In [126]:
# сравниваем ndcg у svd и als с исходными параметрами (!)
user_ndcg_values_als = []
for user_id in df_test['user_id'].unique():
    if user_id < recs_als.shape[0]:
        gt_items = df_test[df_test['user_id'] == user_id]['item_id'].tolist()
        user_recommendations = recs_als[user_id]
        user_ndcg = ndcg_metric(gt_items, user_recommendations)
        user_ndcg_values_als.append(user_ndcg)

avg_ndcg_als = np.mean(user_ndcg_values_als) if user_ndcg_values_als else 0
print(f"ALS NDCG@100: {avg_ndcg_als:.4f}")

user_ndcg_values_svd = []
for user_id in df_test['user_id'].unique():
    if user_id < recs_svd.shape[0]:
        gt_items = df_test[df_test['user_id'] == user_id]['item_id'].tolist()
        user_recommendations = recs_svd[user_id]
        user_ndcg = ndcg_metric(gt_items, user_recommendations)
        user_ndcg_values_svd.append(user_ndcg)

avg_ndcg_svd = np.mean(user_ndcg_values_svd) if user_ndcg_values_svd else 0
print(f"SVD NDCG@100: {avg_ndcg_svd:.4f}")

ALS NDCG@100: 0.5572
SVD NDCG@100: 0.5994


**Вывод**: без подбора гиперпараметров качество у ALS ***хуже***, чем у SVD.

In [131]:
from sklearn.model_selection import ParameterGrid

best_ndcg = 0
best_params = {}

param_grid = {
    'n_factors': [8, 10],
    'alpha': [0.4, 0.5, 0.6],
    'regularization': [0.01, 0.1],
}

for params in ParameterGrid(param_grid): # перебираем гиперпараметры по сетке
    recs_als = make_als_recommendations(coo_interactions, top_k=100, **params)
    user_ndcg_values = [
        ndcg_metric(
            df_test[df_test['user_id'] == user_id]['item_id'].tolist(),
            recs_als[user_id]
        )
        for user_id in df_test['user_id'].unique()
        if user_id < recs_als.shape[0]
    ]
    avg_ndcg = np.mean(user_ndcg_values) if user_ndcg_values else 0
    print(f"Params: {params}, NDCG={avg_ndcg:.4f}")

    if avg_ndcg > best_ndcg:
        best_ndcg = avg_ndcg
        best_params = params

print(f"Лучший NDCG у ALS: {best_ndcg:.4f}, лучшие гиперпараметры: {best_params}")

  0%|          | 0/20 [00:00<?, ?it/s]

Params: {'alpha': 0.4, 'n_factors': 8, 'regularization': 0.01}, NDCG=0.6025


  0%|          | 0/20 [00:00<?, ?it/s]

Params: {'alpha': 0.4, 'n_factors': 8, 'regularization': 0.1}, NDCG=0.6024


  0%|          | 0/20 [00:00<?, ?it/s]

...

Params: {'alpha': 0.6, 'n_factors': 8, 'regularization': 0.1}, NDCG=0.6029


  0%|          | 0/20 [00:00<?, ?it/s]

Params: {'alpha': 0.6, 'n_factors': 10, 'regularization': 0.01}, NDCG=0.5993


  0%|          | 0/20 [00:00<?, ?it/s]

Params: {'alpha': 0.6, 'n_factors': 10, 'regularization': 0.1}, NDCG=0.5993
Лучший NDCG у ALS: 0.6038, лучшие гиперпараметры: {'alpha': 0.6, 'n_factors': 8, 'regularization': 0.01}


**Вывод**: после перебора гиперпараметров NDCG@100 у ALS стал ***выше***, чем у SVD (0.6038 > 0.5994).

##### Задание 3.2. Сделайте объяснение рекомендаций для нескольких юзеров(als.explain). Воспользуйтесь файлом movies.dat чтобы перейти от индексов фильмов к их названием

In [173]:
# для перехода от индексов к названиям фильмов
movies = pd.read_csv("ml-1m/movies.dat", sep='::', engine='python', names=['item_id', 'title', 'genre'], encoding='latin-1')
movies

,item_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [196]:
from typing import List

# для сохранения модели
def create_als_model(
    interactions: Union[np.ndarray, coo_array],
    n_factors: int = 8,
    alpha: float = 0.6,
    regularization: float = 0.01,
):
    if isinstance(interactions, coo_array):
        interactions = csr_matrix(interactions)
    elif isinstance(interactions, np.ndarray):
        interactions = csr_matrix(interactions)
    else:
        raise ValueError("Некорректный тип данных. Ожидается np.ndarray или coo_array")

    als_model = AlternatingLeastSquares(factors=n_factors, regularization=regularization, alpha=alpha, iterations=20, random_state=42)
    als_model.fit(interactions)

    return als_model

als_model = create_als_model(interactions)

recs_als = make_als_recommendations(interactions)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [201]:
def explain_als_recommendations(
    user_ids: List[int],
    recs_als: np.ndarray,
    als_model: AlternatingLeastSquares,
    interactions: csr_matrix,
    movies: pd.DataFrame
):
    explanations = {}
    for user_id in user_ids:
        item_ids = recs_als[user_id]  # 1) выводим индексы рекомендованных фильмов для юзера
        user_explanations = []
        for item_id in item_ids:
            # 2) получаем объяснение для конкретного юзера и фильма
            explanation = als_model.explain(user_id, interactions, itemid=item_id) # explain(userid, user_items, itemid, user_weights=None, N=10)

            # 3) получаем название фильма по его индексу
            movie_data = movies[movies['item_id'] == item_id]
            if not movie_data.empty:  # 4) проверяем, существует ли фильм с таким item_id
                movie_title = movie_data['title'].values[0]
            else:
                continue

            # 5) закидываем в объяснения
            user_explanations.append({
                'item_id': item_id,
                'title': movie_title,
                'explanation': explanation
            })

        user_explanations.sort(key=lambda x: x['explanation'][1], reverse=True)

        explanations[user_id] = user_explanations

    return explanations

user_ids = [1, 350, 1000]
explanations = explain_als_recommendations(user_ids, recs_als, als_model, csr_matrix(interactions), movies)

for user_id, user_explanations in explanations.items():
    print(f"Юзер {user_id}:")
    for exp in user_explanations:
        print(f"  Фильм: {exp['title']}")
        print(f"  Объяснение: {exp['explanation']}")

Юзер 1:
  Фильм: Supercop (1992)
  Объяснение: (0.2644427624625536, [(2788, 0.014193280098202466), (510, 0.013699786598196647), (242, 0.013370710900847198), (892, 0.012803596278126439), (1203, 0.011469198840147184), (47, 0.011308990160841758), (2251, 0.01117799524949248), (2283, 0.011004363772469286), (684, 0.010374546003322507), (891, 0.009837747267425537)], (array([[ 9.16599867e-01, -1.49684177e-01,  5.19340322e-02,
        -4.39886270e-03,  1.78282197e-01, -2.56258723e-02,
         5.20944523e-02,  1.85520095e-02],
       [-1.37200497e-01,  9.65018764e-01, -4.42926664e-03,
         2.07750520e-04, -1.39999951e-01,  6.82571460e-02,
         7.47040731e-03, -1.73896135e-01],
       [ 4.76027270e-02, -1.20480283e-02,  8.47967784e-01,
         8.07801871e-02,  2.10610666e-02,  3.34842392e-02,
         2.47555090e-02,  1.43503570e-01],
       [-4.03199696e-03,  8.58923294e-04,  6.82696254e-02,
         9.17128725e-01, -1.94177901e-02,  8.69241783e-02,
        -1.26641520e-02,  1.25236203

##### Задание 4. До этого мы работали с рейтингами, но как обсуждалось на лекции, implicit ALS отлично работает и с implicit фидбэком. Давайте попробуем преобразовать наш датасет(трейн и тест) следующим образом

1. Бинаризуем все рейтинги(заменим любую интеракцию пользователя на 1)
2. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на 0
3. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1
4. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1 и добавим сглаживание по времени. То есть чем дальше была интеракция от максимальной даты трейна, тем с меньшим весом мы будем ее учитывать(например можно интеракции за последний месяц брать в исходном виде, и с каждым месяцем в прошлое умножать их на какой-нибудь коэффициент меньший 1). Таким образом более старые интеракции пользователя будут вносить меньший вклад в его интересы
5. Придумайте свой вариант(опционально)

Для каждой полученной матрицы обучите iALS и SVD и сравните их результаты между собой(преобразовывать нужно только обучающую выборку, тестовую оставляем неизменной)

In [135]:
# расчёт среднего ndcg
def calc_avg_ndcg(recs, df_test):
    user_ndcg_values = [
        ndcg_metric(
            df_test[df_test['user_id'] == user_id]['item_id'].tolist(),
            recs[user_id]
        )
        for user_id in df_test['user_id'].unique()
        if user_id < recs.shape[0]
    ]
    avg_ndcg = np.mean(user_ndcg_values) if user_ndcg_values else 0
    return avg_ndcg

In [118]:
# ранее в функции была прописана колокна rating, решила добавить в реализацию параметр column, чтобы его можно было менять
def df_to_matrix_upd(df: pd.DataFrame, column: str) -> np.ndarray:
    df = df.copy()

    num_users = df['user_id'].nunique()
    num_items = df['item_id'].nunique()

    result = np.zeros((num_users, num_items), dtype=int)

    for _, row in df.iterrows():
        user_idx = row['user_id']
        item_idx = row['item_id']
        result[user_idx, item_idx] = row[column]

    return result #shape ~ [n_users, n_items]

In [137]:
### 1. Бинаризуем все рейтинги ###
df_train['binary'] = np.where(df_train['rating'] > 0, 1, 0)
interactions_1 = df_to_matrix_upd(df_train, 'binary')

recs_svd = make_svd_recommendations(interactions_1, n_singular_values=10)
recs_als = make_als_recommendations(interactions_1, alpha=0.6, n_factors=8, regularization=0.01)

print(f'NDCG for SVD in option 1: {calc_avg_ndcg(recs_svd, df_test):.4f}')
print(f'NDCG for ALS in option 1: {calc_avg_ndcg(recs_als, df_test):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]

NDCG for SVD in option 1: 0.5863
NDCG for ALS in option 1: 0.5849


**Вывод для преобразования 1**: качество для обоих алгоритмов хуже, чем с исходным рейтингом.

In [138]:
### 2. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на 0 ###
df_train['binary_2'] = np.where(df_train['rating'] >= 4, 1, 0)
interactions_2 = df_to_matrix_upd(df_train, 'binary_2')

recs_svd_2 = make_svd_recommendations(interactions_2, n_singular_values=10)
recs_als_2 = make_als_recommendations(interactions_2, alpha=0.6, n_factors=8, regularization=0.01)

print(f'NDCG for SVD in option 2: {calc_avg_ndcg(recs_svd_2, df_test):.4f}')
print(f'NDCG for ALS in option 2: {calc_avg_ndcg(recs_als_2, df_test):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]

NDCG for SVD in option 2: 0.5708
NDCG for ALS in option 2: 0.5693


**Вывод для преобразования 2**: качество для обоих алгоритмов ещё хуже, чем в преобразовании 1.

In [139]:
### 3. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1 ###
df_train['binary_3'] = np.where(df_train['rating'] >= 4, 1, -1)
interactions_3 = df_to_matrix_upd(df_train, 'binary_3')

recs_svd_3 = make_svd_recommendations(interactions_3, n_singular_values=10)
recs_als_3 = make_als_recommendations(interactions_3, alpha=0.6, n_factors=8, regularization=0.01)

print(f'NDCG for SVD in option 3: {calc_avg_ndcg(recs_svd_3, df_test):.4f}')
print(f'NDCG for ALS in option 3: {calc_avg_ndcg(recs_als_3, df_test):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]

NDCG for SVD in option 3: 0.5829
NDCG for ALS in option 3: 0.5977


**Вывод для преобразования 3**: среди всех преобразований NDCG для ALS здесь наибольшее, однако оно всё ещё не превышает эталонной метрики при исходном рейтинге.

In [160]:
### 4. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1 и добавим сглаживание по времени ###
max_date = df_train['datetime'].max() # максимальная дата в трейне
df_train['time_diff'] = (max_date - df_train['datetime']).dt.days
df_train['weight'] = 0.98 ** (df_train['time_diff'] / 30)

df_train['binary_4'] = np.where(df_train['rating'] >= 4, 1, -1)
df_train['binary_4'] = df_train['binary_4'] * df_train['weight']
interactions_4 = df_to_matrix_upd(df_train, 'binary_4')

recs_svd_4 = make_svd_recommendations(interactions_4, n_singular_values=10)
recs_als_4 = make_als_recommendations(interactions_4, alpha=0.6, n_factors=8, regularization=0.01)

print(f'NDCG for SVD in option 4: {calc_avg_ndcg(recs_svd_4, df_test):.4f}')
print(f'NDCG for ALS in option 4: {calc_avg_ndcg(recs_als_4, df_test):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]

NDCG for SVD in option 4: 0.2802
NDCG for ALS in option 4: 0.2090


**Вывод для преобразования 4**: здесь было достигнуто минимальное качество.

**Решила реализовать своё преобразование [BM25](https://medium.com/@readwith_emma/understanding-okapi-bm25-document-ranking-algorithm-70d81adab001) :)**

По факту код взвешивает взаимодействия юзеров с элементами, придавая больший вес менее популярным элементам

In [172]:
def bm25_weight(data, K1=1.2, B=0.75): # k1  controls the impact of term frequency on the document
    n = len(data) # общее число взаимодействий
    idf = np.log((n - data.value_counts() + 0.5) / (data.value_counts() + 0.5)) # вычисление idf для каждого элемента
    avg_touches = data.value_counts().mean() # cреднее число взаимодействий с одним элементом
    numerator = idf * (K1 + 1) * data.value_counts() # числитель
    denominator = K1 * ((1 - B) + B * (n / avg_touches)) + data.value_counts() # знаменатель
    return numerator / denominator

item_weights = bm25_weight(df_train['item_id']) # вычисляем веса bm25 для каждого item_id
df_train['bm25_weight'] = df_train['item_id'].map(item_weights)
df_train['weighted_rating'] = df_train['rating'] * df_train['bm25_weight'] # рейтинг умножается на соответствующий вес bm25
interactions_5 = df_to_matrix_upd(df_train, 'weighted_rating')

recs_svd_5 = make_svd_recommendations(interactions_5, n_singular_values=10)
recs_als_5 = make_als_recommendations(interactions_5, alpha=0.6, n_factors=8, regularization=0.01)

print(f'NDCG for SVD in option 5: {calc_avg_ndcg(recs_svd_5, df_test):.4f}')
print(f'NDCG for ALS in option 5: {calc_avg_ndcg(recs_als_5, df_test):.4f}')

  0%|          | 0/20 [00:00<?, ?it/s]

NDCG for SVD in option 5: 0.6007
NDCG for ALS in option 5: 0.6111


**Вывод для преобразования 5**: с данным преобразованием было достигнуто лучшее качество для обоих алгоритмов!!

##### Задание 5. iALS на numpy/torch. Давайте реализуем алгоритм iALS на нумпае или торче. Требуется реализовать алгорит, описанный в 4 части [статьи](http://yifanhu.net/PUB/cf.pdf). Обратите внимания на все оптимизации, которые они описывают в статье, чтобы сократить лишние вычисления. Hint: метрики у вашего алгоритма должны быть сравнимы с метриками ALS из библиотеки implicit

In [ ]:
class iALS:
    def __init__(self, n_factors: int = 100, alpha: float = 1.0, reg_coef = 0.01):
        #your code here

    def fit(self, interactions: np.ndarray, n_iterations: int 10):
        #your code here

    def predict(self, top_k: int = 100):
        # возвращает top-k айтемов для каждого юзера(айтемы с которыми юзер взаимодействовал не должны попасть в рекомендации)
        #your code here

        return predicts # shape ~ [n_users, top_k]